In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output

import gymnasium as gym
#from gymnasium.envs.toy_text.frozen_lake import generate_random_map
#random_map = generate_random_map(size=2, p=0.8)

# Example for the Q-Table discussed at length in the Report
env = gym.make('FrozenLake-v1', desc=["SH", "FG"], is_slippery=False , render_mode="rgb_array")

# Example for the 4x4 environment discussed in the performance part of Q-Learning
#env = gym.make('FrozenLake-v1', desc=None, map_name="4x4", is_slippery=True , render_mode="rgb_array")

In [ ]:
""" 
Defaults https://gymnasium.farama.org/environments/toy_text/frozen_lake/

"4x4":[
        "SFFF",
        "FHFH",
        "FFFH",
        "HFFG"
        ]

    "8x8": [
        "SFFFFFFF",
        "FFFFFFFF",
        "FFFHFFFF",
        "FFFFFHFF",
        "FFFHFFFF",
        "FHHFFFHF",
        "FHFFHFHF",
        "FFFHFFFG",
    ]

"""

In [ ]:
# States will be tuples as we are looking at NxN Matrices for our environment

class LakeRunner:
    def __init__(self, learning_rate, epsilon, decay, final_epsilon, gamma = .9):
        # create Q-Table, we need a (B, T)
        self.q_tabel = np.zeros([env.observation_space.n, env.action_space.n])
        self.lr = learning_rate
        self.gamma = gamma
        self.epsilon = epsilon
        self.decay = decay
        self.final_epsilon = final_epsilon
        self.delta = []

    def greedy_pi(self, state):
        # we return the entire row vector of the actions
        if np.random.random() < self.epsilon:
            return env.action_space.sample()
        else:
            return int(np.argmax(self.q_tabel[state]))
    
    # note that reward is actually reward_t+1
    def update(self, state, action, reward, next_state):
        # Splitting this into error and target, so we can track the target

        error =  reward + self.gamma * np.max(self.q_tabel[next_state]) - self.q_tabel[state][action]
        self.q_tabel[state][action] = self.q_tabel[state][action] + self.lr * error
        self.delta.append(error)

    def decay_epsilon(self):
        self.epsilon = max(self.final_epsilon, self.epsilon - self.decay)

In [ ]:
# hyperparameter
learning_rate = 0.01
max_iter = 1000 # set to 2 for minimum needed episodes to be optimal regarding greedy policy
epsilon = 0 # does not matter
decay = 0  # does not matter
final_epsilon = 0 # does not matter

agent = LakeRunner(learning_rate, epsilon, decay, final_epsilon)

# Running the 2x2 example, on the fixed Episode
# The numbering for the actions is as follows
# left: 0, down: 1, right: 2, up: 3
# Sequenz = [(0,3), (0,1), (2,0), (2,2)] G, the terminal state is missing as we end there
# However, the env is setup to know the state, so we only need to pass the action to take
sequenz = [3, 1, 0, 2] 

for episode in range(max_iter):
    state, info = env.reset()
    clear_output()
    for action in sequenz:
        next_state, reward, terminated, truncated, info = env.step(action)
        
        # update the agent
        agent.update(state, action, reward, next_state)
        frame = env.render()
        plt.imshow(frame)
        plt.show()
        # update if the environment is done and the current state
        state = next_state

In [ ]:
# hyperparameter
learning_rate = 0.01
max_iter = 1
epsilon = 1.0
decay = epsilon / (max_iter/ 2)  # reduce the exploration over time
final_epsilon = 0.1

agent = LakeRunner(learning_rate, epsilon, decay, final_epsilon)

In [ ]:
env = gym.wrappers.RecordEpisodeStatistics(env, deque_size=max_iter)
env.reset()
sequenz = []
for episode in range(max_iter):
    state, info = env.reset()
    done = False
    clear_output()

    # play one episode
    while not done:
        action = agent.greedy_pi(state)
        sequenz.append(action)
        next_state, reward, terminated, truncated, info = env.step(action)
        
        # update the agent
        agent.update(state, action, reward, next_state)
        frame = env.render()
        plt.imshow(frame)
        plt.show()
        # update if the environment is done and the current state
        done = terminated or truncated
        state = next_state

    agent.decay_epsilon()

In [ ]:
rolling_length = 500
fig, axs = plt.subplots(ncols=3, figsize=(12, 5))
axs[0].set_title("Episode rewards")
reward_moving_average = (
    np.convolve(
        np.array(env.return_queue).flatten(), np.ones(rolling_length), mode="valid"
    )
    / rolling_length
)
axs[0].plot(range(len(reward_moving_average)), reward_moving_average)
axs[1].set_title("Episode lengths")
length_moving_average = (
    np.convolve(
        np.array(env.length_queue).flatten(), np.ones(rolling_length), mode="same"
    )
    / rolling_length
)
axs[1].plot(range(len(length_moving_average)), length_moving_average)
axs[2].set_title("Training Error")
training_error_moving_average = (
    np.convolve(np.array(agent.delta), np.ones(rolling_length), mode="same")
    / rolling_length
)
axs[2].plot(range(len(training_error_moving_average)), training_error_moving_average)
plt.tight_layout()
plt.show()